##### **Disclaimer: We use some advanced packages here without detailed explanation. You can use these, but we do not provide any support.**

In [1143]:
# To install them, you can uncomment the following lines:
# (%pip will call pip from the currently active python environment)

# Note: Some of these packages are still not compatible with Python 3.12 yet
# %pip install sweetviz
# %pip install ydata_profiling
# %pip install shap

## <font style="font-weight: bold;"> Analytics Cup 2024 </font>

In [1144]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

# Note: The following do not work with Python 3.12
import shap
from ydata_profiling import ProfileReport
import sweetviz as sv

#### Reproducibility

In [1145]:
seed = 2024

# pandas, statsmodels, matplotlib and y_data_profiling rely on numpy's random generator, and thus, we need to set the seed in numpy
np.random.seed(seed)

### <font color='green'> Phase 1: Business Understanding </font>

Business Understanding is the first and economically most important step in the
CRISP-DM process. It serves to assess use cases, feasibility, requirements, and
risks of the endeavored data driven project. Since the conduction of data driven
projects usually depends on the data at hand, the CRISP-DM process often 
alternates between Business Understanding and Data Understanding, until the
project's schedule becomes sufficiently clear.

#### Business Understanding

In LLMeals Analytics Cup, the goal is to improve customer satisfaction and reduce subscription cancellations by developing a model that accurately predicts whether a customer likes (Like=1) or dislikes (Like=0) a suggested recipe. The model will utilize datasets such as "recipes.csv," "reviews.csv," "diet.csv," and "requests.csv" to generate insights into user preferences. The successful model will serve as the foundation for enhancing the quality of suggested recipes in LLMeals' service, aligning them more closely with individual customer requirements. The project's ultimate aim is to leverage data-driven approaches for refining the recipe suggestions and, in turn, improving the overall LLMeals user experience.

### <font color='green'> Phase 2: Data Understanding </font>

The *Data Understanding* phase mainly serves to inform the Business Understanding step by
assessing the data quality and content, and should provide the engineers with 
an intuition for the specific data and the specific problem at hand. Experienced
data scientists and machine learning engineers can often estimate the difficulty
and feasibility of the task by analyzing and understanding the data.  

#### Example: Data Understanding

Make yourself familiar with the structure and content of the data. *Note*, this step 
heavily depends on the specific problem at hand, since there is no fixed recipe that 
fits all possible data sets. In the example below, we are only looking at a very small
data set and do **not** conduct an in-depth analysis.  

In [1146]:
# load the data
file_dir = "datasets/original"
file_names = ["reviews.csv", "requests.csv", "diet.csv", "recipes.csv"]
reviews = pd.read_csv(f'{file_dir}/{file_names[0]}', low_memory=False)
requests = pd.read_csv(f'{file_dir}/{file_names[1]}')
diet = pd.read_csv(f'{file_dir}/{file_names[2]}')
recipes = pd.read_csv(f'{file_dir}/{file_names[3]}')

Reviews

In [1147]:
# print(reviews.sample(3))
# print("\n")
# print(reviews.info())
# print("\n")
# print(reviews.describe())
# sns.boxplot(reviews)

Requests

In [1148]:
# print(requests.sample(3))
# print("\n")
# print(requests.info())
# print("\n")
# print(requests.describe())
# sns.boxplot(requests)

Diet

In [1149]:
# print(diet.sample(3))
# print("\n")
# print(diet.info())
# print("\n")
# print(diet.describe())
# sns.boxplot(diet)

Recipes

In [1150]:
# print(recipes.sample(3))
# print("\n")
# print(recipes.info())
# print("\n")
# print(recipes.describe())
# plt.figure(figsize=(24, 6))
# sns.boxplot(recipes)

#### Class Balance

In [1151]:
# # check the balancing of classes/labels
# print(reviews.groupby("Like").size())

# # -> 2 classes, 1 is much more frequent than the other (False/True ratio is ~4:1)

#### Reviews Feature Distribution

In [1152]:
# have a look at the feature distributions with a pairplot,
# as it gives you a good overview over possible outliers
# and a good overview over the data in general

# pairplot for the full data
# columns_to_drop = ["AuthorId", "RecipeId", "TestSetId"]
# sns.pairplot(reviews.drop(columns_to_drop, axis=1), hue="Like", diag_kind="hist", diag_kws={"multiple" : "stack"})

#### Requests Feature Distribution

In [1153]:
# have a look at the feature distributions with a pairplot,
# as it gives you a good overview over possible outliers
# and a good overview over the data in general

# pairplot for the full data
# columns_to_drop = ["AuthorId", "RecipeId"]
# data = pd.merge(requests, reviews[["AuthorId", "RecipeId", "Like"]], on=['AuthorId','RecipeId'], \
#     how='left').drop(columns_to_drop, axis=1)
# sns.pairplot(data, hue="Like", diag_kind="hist", diag_kws={"multiple" : "stack"})

#### Diet Feature Distribution

In [1154]:
# pairplot for the full data
# columns_to_drop = ["AuthorId"]
# data = pd.merge(diet, reviews[["AuthorId", "Like"]], on=['AuthorId'], how='left').drop(columns_to_drop, axis=1)
# sns.pairplot(data, hue="Like", diag_kind="hist", diag_kws={"multiple" : "stack"})

#### Recipes Feature Distribution

In [1155]:
# pairplot for the full data
# columns_to_drop = ["RecipeId"]
# data = pd.merge(recipes, reviews[["RecipeId", "Like"]], on=['RecipeId'], how='left').drop(columns_to_drop, axis=1)
# sns.pairplot(data, hue="Like", diag_kind="hist", diag_kws={"multiple" : "stack"})

### Create a merged dataset

| **File**     | **Join Keys**            |
|--------------|------------------------|
| reviews.csv  | AuthorId, RecipeId     |
| requests.csv | AuthorId, RecipeId     |
| diet.csv     | AuthorId               |
| recipes.csv  | RecipeId               |


In [1156]:
# # Merge the dataframes using multiple columns
# merged_df = pd.merge(reviews, requests, on=['AuthorId','RecipeId'], how='left')
# merged_df = pd.merge(merged_df, diet, on='AuthorId', how='left')
# merged_df = pd.merge(merged_df, recipes, on='RecipeId', how='left')

# # Save the merged dataframe to a new CSV file
# merged_df.to_csv('datasets/original/merged_data.csv', index=False)

# # Check if the number of rows and columns are correct
# print(len(merged_df) == len(reviews))
# print(reviews.shape)
# print(merged_df.shape)
# print(merged_df.shape[1] == reviews.shape[1] + requests.shape[1]-2 + diet.shape[1]-1 + recipes.shape[1]-1)

### Import merged dataset

In [1157]:
# load the data
file_dir = "datasets/original"
file_name = "merged_data.csv"
df = pd.read_csv(f'{file_dir}/{file_name}', low_memory=False)
index_column = df.columns[0]
df.drop([index_column], axis=1, inplace=True)

#### Class-dependent pairplots

In [1158]:
# # look at class-dependent pairplots
# _df = df.drop(["AuthorId", "RecipeId", "TestSetId"], axis=1, inplace=False)
# df_grouped_by_class = _df.groupby(by="Like")

# df_true = df_grouped_by_class.get_group(True)
# df_false = df_grouped_by_class.get_group(False)

# class_labels = {
#     "True" : {
#         "color" : "blue",
#         "data" : df_true
#     },
#     "False" : {
#         "color" : "green",
#         "data" : df_false
#     }
# }

# for class_i in class_labels:
#     class_color = class_labels[class_i]["color"]
#     class_df = class_labels[class_i]["data"]
#     p = sns.pairplot(class_df, diag_kind="hist", diag_kws={"color" : class_color}, plot_kws={"color" : class_color, "label" : class_i})
#     p.fig.suptitle(class_i, y=1.0, size=15)

### Summary Report

In [1159]:
# # We can also leverage the dataprep package to get a nice summary report
# report = sv.analyze(df)
# report.show_notebook()

# # We can also leverage the yadata_profiling package to get a nice summary report
# profile = ProfileReport(df, title="LLMeals - Summary Report")
# profile

#### Summary: Data Understanding

You should have a good understanding what the data is about and of some of its properties. Newly gained insights are used to reiterate the
Business Understanding Phase, but in this example, it won't be necessary.

### <font color='green'> Phase 3: Data Preparation </font>

Data Preparation mainly consists of two parts, Data Cleaning and Data Wrangling. In Data
Cleaning, the goal is assure data quality. This includes removing wrong/corrupt 
data entries and making sure the entries are standardized, e.g. enforcing certain encodings. 
Data Wrangling then transforms the data in order to make it suitable for the modelling step.
Sometimes, steps from Data Wrangling are incorporated into the automatized Pipeline, as
we will show in this example.

#### Data Cleaning

Variable Encoding

In [1160]:
from sklearn.preprocessing import LabelEncoder

file_source_path = 'datasets/original/merged_data.csv' # source file
file_dir = 'datasets/encoded' # destination directory
file_tag = 'dataset'

df = pd.read_csv(file_source_path, low_memory=False)
index_column = df.columns[0]
df.drop([index_column], axis=1, inplace=True)

# Like: object -> bool
# HighProtein: {Indiferent, Yes} - object -> bool
# LowSugar: {0, Indiferent} - object -> bool
# Diet: {Vegetarian, Omnivore, Vegan} - object -> categorical
# Name: 140k values, 50% distinct - object -> categorical
# RecipeCategory: 7 categories - object -> categorical
# RecipeIngredientQuantities: filter
# RecipeIngredientParts: filter
# RecipeYield: 46k values, 7.9k distinct, 93.8k missing - object -> categorical


# --------------------------------------------------------- #
# IMPORTANT: call this function after loading the dataframe #
# --------------------------------------------------------- #
def convert_variable_types(df: DataFrame):
    for c in df.columns:
        uniques = df[c].dropna(inplace=False).unique()
        if len(uniques) == 2:
            df[c] = df[c].astype('bool')
        elif df[c].dtype == 'int64' or df[c].dtype == 'float64':
            continue # do nothing, already correct type
        else:
            df[c] = df[c].astype('category')

# ----------- Convertions ----------- #
# df["Like"] = df["Like"].astype('category')
# df["HighProtein"] = df["HighProtein"].astype('category')
# df["LowSugar"] = df["LowSugar"].astype('category')
# df["Diet"] = df["Diet"].astype('category')
# df["Name"] = df["Name"].astype('category')
# df["RecipeCategory"] = df["RecipeCategory"].astype('category')
# df["RecipeYield"] = df["RecipeYield"].astype('category')
# df["RecipeIngredientQuantities"] = df["RecipeIngredientQuantities"].astype('category')
# df["RecipeIngredientParts"] = df["RecipeIngredientParts"].astype('category')
# df["AuthorId"] = df["AuthorId"].astype('category')
# Note: "Rating" was kept as float64 to allow for decimal values

# convert all variables to the correct type
convert_variable_types(df)

# save the categorical variables
categorical_vars = [column for column in df.columns if df[column].dtype.name == 'category']
print("Categorical vars: ", categorical_vars)

# save the boolean variables
bool_vars = [column for column in df.columns if df[column].dtype.name == 'bool']
print("Boolean vars: ", bool_vars)

# save the non-numeric variables
non_numeric_vars = categorical_vars + bool_vars
print("Non-numeric vars: ", non_numeric_vars)

# convert categorical variables to numerical
for c in categorical_vars:
    le = LabelEncoder()
    df[c] = le.fit_transform(df[c])

df.to_csv(f'{file_dir}/{file_tag}_encoded.csv', index=True)

# ---------------------------------------------

# # filter RecipeIngredientQuantities
# import re
# from fractions import Fraction

# # Function to convert a string to a list of floats with fractions
# def convert_to_float_array(value):
#     clean_values = value.replace('c(', '').replace('"', '').replace('\\', '').replace(')', '').split(',')
#     result = []
#     for item in clean_values:
#         # Skip empty strings
#         if item:
#             # Check for mixed fractions with a space before them
#             if ' ' in item:
#                 whole_part, fraction_part = item.split(' ')
#                 result.append(float(whole_part) + float(Fraction(fraction_part)))
#             else:
#                 result.append(float(Fraction(item)))
#     return result

# # df['RecipeIngredientQuantities'] = df['RecipeIngredientQuantities'].astype(str)
# print(df['RecipeIngredientQuantities'].head())
# # print(x for x in df.iloc[0]['RecipeIngredientQuantities'])

# df['RecipeIngredientQuantities'] = df['RecipeIngredientQuantities'].apply(convert_to_float_array)

# # Flatten the lists and remove None values
# # df['RecipeIngredientQuantities'] = df['RecipeIngredientQuantities'].apply(lambda x: item for sublist in x \
# #                                                         if sublist is not None and sublist != [] for item in sublist)

# df['RecipeIngredientQuantities'].head()


Categorical vars:  ['Diet', 'Name', 'RecipeCategory', 'RecipeIngredientQuantities', 'RecipeIngredientParts', 'RecipeYield']
Boolean vars:  ['Like', 'HighCalories', 'HighProtein', 'LowFat', 'LowSugar', 'HighFiber']
Non-numeric vars:  ['Diet', 'Name', 'RecipeCategory', 'RecipeIngredientQuantities', 'RecipeIngredientParts', 'RecipeYield', 'Like', 'HighCalories', 'HighProtein', 'LowFat', 'LowSugar', 'HighFiber']


Missing values

In [1172]:
# fill/remove/change missing/corrupt values
from pandas import concat, DataFrame
from sklearn.impute import SimpleImputer
from numpy import nan

file_source_path = 'datasets/encoded/dataset_encoded.csv' # source file
file_dir = 'datasets/missing_values' # destination directory
file_tag = 'dataset'

# Import data
df = pd.read_csv(file_source_path, low_memory=False)
convert_variable_types(df)
index_column = df.columns[0]
df.drop([index_column], axis=1, inplace=True)

# --------------- #
# Missing Values  #
# --------------- #

print("Missing values:")
for var in df:
    nr = df[var].isna().sum()
    if nr > 0:
        print(f"{var} : {nr} ({round(nr/df[var].shape[0]*100, 2)}%)")

Missing values:
Rating : 63087 (45.0%)
TestSetId : 97381 (69.46%)
RecipeServings : 50021 (35.68%)


In [ ]:
# defines the number of records to discard entire COLUMNS
threshold = df.shape[0] * 0.90

# drop columns with more missing values than the defined threshold
missings = [c for c in mv.keys() if mv[c]>threshold]
df = df.drop(columns=missings, inplace=False)

In [1162]:
# AUX: Get variable types
def get_variable_types(df: DataFrame) -> dict:
    variable_types: dict = {
        'Numeric': [],
        'Binary': [],
        'Categorical': []
    }

    for c in df.columns:
        if df[c].dtype == 'bool':
            variable_types['Binary'].append(c)
        elif df[c].dtype == 'int64' or df[c].dtype == 'float64':
            variable_types['Numeric'].append(c)
        elif df[c].dtype == 'category':
            variable_types['Categorical'].append(c)
        else:
            print(f'Unknown variable type for {c}')

    return variable_types

In [1163]:
# -------------------------------------------------------------- #
# APPROACH 1: Fill with CONSTANT Value after DROP Missing Values #
# -------------------------------------------------------------- #

# AUX: Fill with CONSTANT value
def fill_with_constant(data: DataFrame) -> DataFrame:
    tmp_nr, tmp_cat, tmp_bool = None, None, None
    variables = get_variable_types(data)
    numeric_vars = variables['Numeric']
    categorical_vars = variables['Categorical']
    binary_vars = variables['Binary']

    if len(numeric_vars) > 0:
        imp = SimpleImputer(strategy='constant', fill_value=0, missing_values=nan, copy=True)
        tmp_nr = DataFrame(imp.fit_transform(data[numeric_vars]), columns=numeric_vars)
    if len(categorical_vars) > 0:
        imp = SimpleImputer(strategy='constant', fill_value=-1, missing_values=nan, copy=True)
        tmp_cat = DataFrame(imp.fit_transform(data[categorical_vars]), columns=categorical_vars)
    if len(binary_vars) > 0:
        imp = SimpleImputer(strategy='constant', fill_value=False, missing_values=nan, copy=True)
        tmp_bool = DataFrame(imp.fit_transform(data[binary_vars].astype(int)), columns=binary_vars).astype(bool)

    df = concat([tmp_nr, tmp_cat, tmp_bool], axis=1)
    df.index = data.index

    return df

# ----------------------------------------------------------------- #

# Fill the rest with constant
df_const = fill_with_constant(df)
df_const.to_csv(f'{file_dir}/{file_tag}_drop_columns_then_constant.csv', index=True)
# df_const.head()

# Best results: Score = 0.533

In [1164]:
# -------------------------------------------------------------- #
# APPROACH 2: Fill with CONSTANT Value after DROP Missing Values #
# -------------------------------------------------------------- #

# AUX: Fill with MOST FREQUENT value
def fill_with_most_frequent(data: DataFrame) -> DataFrame:
    tmp_nr, tmp_cat, tmp_bool = None, None, None
    variables = get_variable_types(data)
    numeric_vars = variables['Numeric']
    categorical_vars = variables['Categorical']
    binary_vars = variables['Binary']

    tmp_nr, tmp_cat, tmp_bool = None, None, None
    if len(numeric_vars) > 0:
        imp = SimpleImputer(strategy='mean', missing_values=nan, copy=True)
        tmp_nr = DataFrame(imp.fit_transform(data[numeric_vars]), columns=numeric_vars)
    if len(categorical_vars) > 0:
        imp = SimpleImputer(strategy='most_frequent', missing_values=nan, copy=True)
        tmp_cat = DataFrame(imp.fit_transform(data[categorical_vars]), columns=categorical_vars)
    if len(binary_vars) > 0:
        imp = SimpleImputer(strategy='most_frequent', missing_values=nan, copy=True)
        tmp_bool = DataFrame(imp.fit_transform(data[binary_vars].astype(int)), columns=binary_vars).astype(bool)

    df = concat([tmp_nr, tmp_cat, tmp_bool], axis=1)
    df.index = data.index

    return df

# ----------------------------------------------------------------- #


# Fill the rest with most frequent value
df_most_freq = fill_with_most_frequent(df)
df_most_freq.to_csv(f'{file_dir}/{file_tag}_drop_columns_then_most_frequent.csv', index=True)
# df_most_freq.head()

# Best results: Score = 0.533

In [1165]:
# TODO: Explore more approaches to fill missing values

#### Data Wrangling

In contrast to Data Cleaning, Data Wrangling _transforms_ the dataset, in order
to prepare it for the training of the models. This includes scaling, dimensionality
reduction, data augmentation, outlier removal, etc.

Outliers treatment

In [1166]:
from scipy.stats import shapiro

# Best option
file_source_path = 'datasets/missing_values/dataset_drop_columns_then_most_frequent.csv' # source file
file_dir = 'datasets/outliers' # destination directory
file_tag = 'dataset'

# read the data
df = pd.read_csv(file_source_path, low_memory=False)
convert_variable_types(df)
index_column = df.columns[0]
df.drop([index_column], axis=1, inplace=True)

# non_numeric_vars =  ['Diet', 'Name', 'RecipeCategory', 'RecipeIngredientQuantities', 'RecipeIngredientParts', \
#                      'RecipeYield', 'Like', 'HighCalories', 'HighProtein', 'LowFat', 'LowSugar', 'HighFiber']

numeric_vars = get_variable_types(df)['Numeric']
# remove original non-numeric variables 
for var in numeric_vars.copy():
    if var in non_numeric_vars:
        numeric_vars.remove(var)

# check for variables that are normally distributed
norm_dist_variables = []
for var in numeric_vars:
    stat, p_value = shapiro(df[var])
    # Interpret the result
    alpha = 0.05
    if p_value > alpha:
        # The variable looks normally distributed (fail to reject H0)
        norm_dist_variables.append(var)

print('Original dataset:', df.shape)
print('Normal distributed variables:', norm_dist_variables)
# "Rating" is the only normally distributed variable but it only has 1 unique value in the dataset -> ignore it
summary5 = df.describe(include='number')

p-value may not be accurate for N > 5000.
Input data for shapiro has range zero. The results may not be accurate.


Original dataset: (140195, 29)
Normal distributed variables: ['Rating']


In [1167]:
def determine_outlier_thresholds(summary5: DataFrame, var: str, OPTION: str, OUTLIER_PARAM: int):
    # default parameter
    if OPTION == 'iqr':
        iqr = OUTLIER_PARAM * (summary5[var]['75%'] - summary5[var]['25%'])
        top_threshold = summary5[var]['75%'] + iqr
        bottom_threshold = summary5[var]['25%'] - iqr
    # for normal distribution
    elif OPTION == 'stdev':
        std = OUTLIER_PARAM * summary5[var]['std']
        top_threshold = summary5[var]['mean'] + std
        bottom_threshold = summary5[var]['mean'] - std
    else:
        raise ValueError('Unknown outlier parameter!')
    return top_threshold, bottom_threshold

In [1168]:
# ------------------------- #
# APPROACH 1: Drop outliers #
# ------------------------- #

# Tuned parameter to get the better results
IQR_PARAM = 30

data = df.copy(deep=True)

for var in numeric_vars:
    top_threshold, bottom_threshold = determine_outlier_thresholds(summary5, var, 'iqr', IQR_PARAM)
    outliers = data[(data[var] > top_threshold) | (data[var] < bottom_threshold)]
    print(f'{var} outliers: {outliers.shape[0]}/{data[var].shape[0]}')
    data.drop(outliers.index, axis=0, inplace=True)
data.to_csv(f'datasets/outliers/{file_tag}_drop_outliers.csv', index=True)
print('Dataset after dropping outliers:', data.shape)

# Best results: Score = 0.803

# IQR_PARAM results:
#    IQR    |   20    |   25    |   30    |   35    |   40    |
# --------- |---------|---------|---------|---------|---------|
#   Score   |  0.803  |  0.803  |  0.803  |  0.803  |  0.803  |

RecipeId outliers: 0/140195
Rating outliers: 0/140195
TestSetId outliers: 42814/140195
Time outliers: 436/97381
Age outliers: 0/96945
CookTime outliers: 212/96945
PrepTime outliers: 890/96733
Calories outliers: 39/95843
FatContent outliers: 8/95804
SaturatedFatContent outliers: 8/95796
CholesterolContent outliers: 10/95788
SodiumContent outliers: 109/95778
CarbohydrateContent outliers: 56/95669
FiberContent outliers: 10/95613
SugarContent outliers: 274/95603
ProteinContent outliers: 1/95329
RecipeServings outliers: 31/95328
Dataset after dropping outliers: (95297, 29)


In [1169]:
# ----------------------------- #
# APPROACH 2: Truncate outliers #
# ----------------------------- #

# Tuned parameter to get the better results
IQR_PARAM = 20

data = df.copy(deep=True)

for var in numeric_vars:
    top_threshold, bottom_threshold = determine_outlier_thresholds(summary5, var, 'iqr', IQR_PARAM)
    original_column = data[var].copy()
    data[var] = data[var].apply(lambda x: top_threshold if x > top_threshold else bottom_threshold if x < bottom_threshold else x)
    # print(f'{var} outliers: {(data[var] != original_column).sum()}/{data[var].shape[0]}')
data.to_csv(f'datasets/outliers/{file_tag}_truncate_outliers_{IQR_PARAM}.csv', index=True)
print('Dataset after truncating outliers:', data.shape)
    
# Best results: Score = 0.803

# IQR_PARAM results:
#    IQR    |   20    |   25    |   30    |   35    |   40    |
# --------- |---------|---------|---------|---------|---------|
#   Score   |  0.803  |  0.803  |  0.803  |  0.803  |  0.512  |

Dataset after truncating outliers: (140195, 29)


In [1170]:
### data wrangling

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, MinMaxScaler, Z-score

# data scaling
transform_scaler = StandardScaler()

# dimensionality reduction
transform_pca = PCA()

# value imputing

# outlier detection/removal

SyntaxError: invalid syntax (1084077941.py, line 4)

Outliers

#### Sampling

Once we have a cleaned data set, and before we start the *Modelling* phase, we are going to split our data set into multiple sub-datasets. 
Here, we are going to split it into an *train* and *test* data set. Note that the split strongly depends on the underlying use-case
and used dataset.  

In [ ]:
# split data into learning and test sets
from sklearn.model_selection import train_test_split

file_source_path = 'datasets/outliers/BEST_OUTLIERS.csv' # source file

# read data
df = pd.read_csv(f'{file_source_path}', low_memory=False)
# convert variable types
convert_variable_types(df)
# remove index column
index_column = df.columns[0]
df = df.drop([index_column], axis=1)
# Drop TestSetId column and Like NaN rows (no way to know if they liked or not)
df.drop('TestSetId', axis=1, inplace=True)
df.dropna(subset=[target], inplace=True)

y = df.pop(target).values
X = df.values

X_train, X_test, y_train, y_test = \
    train_test_split(X, y,
                test_size=0.3, 
                shuffle=True,
                random_state=3)


### <font color='green'> Phase 4: Modeling </font>

In this phase, the model is trained and tuned. In general, data transformations
from data wrangling can be part of a machine learning pipeline, and can therefore
be tuned as well. (See CRISP-DM: DataPrep <--> Modeling)

In [ ]:
# Here, you want to find the best classifier. As candidates, consider
#   1. LogisticRegression
#   2. RandomForestClassifier
#   3. other algorithms from sklearn (easy to add)
#   4. custom algorithms (more difficult to implement)
    
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

model_logistic_regression = LogisticRegression(max_iter=30)
model_random_forest = RandomForestClassifier()
model_gradient_boosting = GradientBoostingClassifier()

# train the models
pipeline = Pipeline(steps=[("scaler", transform_scaler), 
                           ("pca", transform_pca),
                           ("model", None)])

parameter_grid_preprocessing = {
  # "pca__n_components" : [1, 2, 3, 4],
  "pca__n_components" : [df.shape[1]-12, df.shape[1]-8, df.shape[1]-5, df.shape[1]]
}

parameter_grid_logistic_regression = {
  "model" : [model_logistic_regression],
  "model__C" : [0.1, 1, 10],  # inverse regularization strength
}

parameter_grid_gradient_boosting = {
  "model" : [model_gradient_boosting],
  "model__n_estimators" : [10, 20, 30]
}

parameter_grid_random_forest = {
  "model" : [model_random_forest],
  "model__n_estimators" : [10, 20, 50],  # number of max trees in the forest
  "model__max_depth" : [2, 3, 4],
}

meta_parameter_grid = [parameter_grid_logistic_regression,
                       parameter_grid_random_forest,
                       parameter_grid_gradient_boosting]

meta_parameter_grid = [{**parameter_grid_preprocessing, **model_grid}
                       for model_grid in meta_parameter_grid]

search = GridSearchCV(pipeline,
                      meta_parameter_grid, 
                      scoring="balanced_accuracy",
                      n_jobs=2, 
                      cv=5,  # number of folds for cross-validation 
                      error_score="raise"
)

# here, the actual training and grid search happens
search.fit(X_train, y_train.ravel())

print("best parameter:", search.best_params_ ,"(CV score=%0.3f)" % search.best_score_)

ValueError: Input X contains NaN.
PCA does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

### <font color='green'> Step 5: Evaluation </font>

Once the appropriate models are chosen, they are evaluated on the test set. For
this, different evaluation metrics can be used. Furthermore, this step is where
the models and their predictions are analyzed resp. different properties, including
feature importance, robustness to outliers, etc.

In [ ]:
# evaluate performance of model on test set
print("Score on test set:", search.score(X_test, y_test.ravel()))

# contingency table
ct = pd.crosstab(search.best_estimator_.predict(X_test), y_test.ravel(),
                 rownames=["pred"], colnames=["true"])
print(ct)

In [ ]:
# (optional, if you're curious) 
# for a detailed look on the performance of the different models
def get_search_score_overview():
  for c,s in zip(search.cv_results_["params"],search.cv_results_["mean_test_score"]):
      print(c, s)

print(get_search_score_overview())

#### Interpretability

##### Disclaimer: This only works if shap is installed.

In addition to models and their predictions, it is often important to understand _why_ a model makes certain predictions. 
There is a lot of literature on how this can be achieved (explainability), but we will only show the use of Shapley values
using the python module "shap", which is a combination of Shapley values and LIME. 
You can find more information on this topic [here](https://christophm.github.io/interpretable-ml-book/shap.html).

In [ ]:
# assume random forest model
model = RandomForestClassifier(n_estimators=10, random_state=seed)
model.fit(X_train, y_train.values.ravel())

# compute shapley values
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_train)
shap_interaction_values = explainer.shap_interaction_values(X_train)

expected_value = explainer.expected_value
print(expected_value)

In [ ]:
# class dependent plots of shapley values for each feature
for i,c in enumerate(df.variety.unique()):
    shap.summary_plot(shap_values[i], X_train, show=False)
    plt.title("Shapley values for "+str(c))
    plt.show()

From the computed SHAP values, we can interpret that the *petal.width* has a positive impact on the output of the model 
if the feature value is moderate. For high aand low values, the impact is negative. The same observation
holds for *petal.length*. Besides, the impact of the *sepal.length* and *sepal.width* features are rather low. By impact on a 
the target, we model the probability that we classify that target. Thus, if *petal.width* is high, it is more likely
that we classify the data point as Versicolor.

### <font color='green'> Step 6: Deployment </font>

Now that you have chosen and trained your model, it is time to deploy it to your
clients system. 

In [ ]:
def micro_service_classify_iris(datapoint):
    
  # make sure the provided datapoints adhere to the correct format for model input

  # fetch your trained model
  model = search.best_estimator_

  # make prediction with the model
  prediction = model.predict(datapoint)

  return prediction


In [ ]:
# hypothetical new batch of flowers arrives
from scipy.stats import norm

amount_of_new_flowers = 9
df_flowers = pd.DataFrame(columns=df.columns.drop("variety"), index=range(1, amount_of_new_flowers+1))

for i in df_flowers.index:
  df_flowers.loc[i, "sepal.length"] = norm(loc=6, scale=2).rvs()
  df_flowers.loc[i, "sepal.width"] = norm(loc=3, scale=1).rvs()
  df_flowers.loc[i, "petal.length"] = norm(loc=3, scale=5).rvs()
  df_flowers.loc[i, "petal.width"] = norm(loc=2, scale=2).rvs()

# customer uses your micro service to determine the varieties
df_flowers["variety"] = micro_service_classify_iris(df_flowers)
print(df_flowers)

In the Analytics Cup, you need to export your prediction in a very specific output format. This is a csv file without an index and two columns, *id* and *prediction*. Note that the values in both columns need to be integer values, and especially in the *prediction* column either 1 or 0.

In [ ]:
# Let's assume that our id column is the index of the dataframe
output = pd.DataFrame(df_flowers.variety)
output['id'] = df_flowers.index
output = output.rename(columns={'variety': 'prediction'})
output = output.reindex(columns=["id", "prediction"])
output.to_csv('iris_prediction.csv', index=False)